# Using K8s and Kserve python modules

In [24]:
import kfp
from kfp import components
from kfp.dsl import component, pipeline, Input, Output, Dataset, Model, Metrics, ClassificationMetrics, OutputPath
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get GitHub username and token from environment variables
GITHUB_USERNAME = os.getenv("GITHUB_USERNAME")
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")

# Define configuration variables
REPO_URL = "https://github.com/danilonicioka/mlops-workflow.git"
CLONED_DIR = "mlops-workflow"
BRANCH_NAME = "tests"
PIPELINE_ID = "my-pipeline-id"
PIPELINE_NAME = "mlops"
KFP_HOST = "http://ml-pipeline.kubeflow:8887"  # KFP host URL

# Define DVC remote configuration variables
REMOTE_NAME = "minio_remote"
REMOTE_URL = "s3://dvc-data"
MINIO_URL = "http://minio-service.kubeflow:9000"
ACCESS_KEY = os.getenv("ACCESS_KEY")
SECRET_KEY = os.getenv("SECRET_KEY")
DVC_FILE_DIR = 'data/external'
DVC_FILE_NAME = 'dataset.csv'

# Model serve config vars
MODEL_NAME = "youtubegoes5g"
FRAMEWORK = "pytorch"
NAMESPACE = "kubeflow-user-example-com"
SVC_ACCOUNT = "default-editor"
MODEL_URI = "pvc://model-store-claim"

@component(base_image="python:3.11.9", packages_to_install=['kserve==0.13.0','kubernetes==30.1.0'])
def model_serving(
    model_name: str,
    namespace: str,
    model_uri: str
):
    # Create kserve instance
    from kubernetes import client 
    from kserve import KServeClient, constants, V1beta1InferenceService, V1beta1InferenceServiceSpec, V1beta1PredictorSpec, V1beta1TorchServeSpec
    from datetime import datetime
    import time
    from subprocess import run
    
    #TFServing wants this type of structure ./models/1/model
    # the number represent the model version
    # in this example we use only 1 version
    
    #Inference server config
    now = datetime.now()
    kserve_version='v1beta1'
    api_version = constants.KSERVE_GROUP + '/' + kserve_version

    isvc = V1beta1InferenceService(api_version=api_version,
                                   kind=constants.KSERVE_KIND,
                                   metadata=client.V1ObjectMeta(
                                       name=model_name, namespace=namespace, annotations={'sidecar.istio.io/inject':'false'}),
                                   spec=V1beta1InferenceServiceSpec(
                                   predictor=V1beta1PredictorSpec(
                                       service_account_name="default-editor",
                                       pytorch=(V1beta1TorchServeSpec(
                                           storage_uri=model_uri))))
    )

    KServe = KServeClient()
    
    #replace old inference service with a new one
    try:
        KServe.delete(name=model_name, namespace=namespace)
        print("Old model deleted")
    except:
        print("Couldn't delete old model")
    time.sleep(10)
    
    KServe.create(isvc)

@pipeline
def my_pipeline(
    model_name: str,
    namespace: str,
    model_uri: str
):
    model_serving_task = model_serving(namespace=namespace, 
                                       model_name=model_name,
                                       model_uri=model_uri)

# Compile the pipeline
pipeline_filename = f"{PIPELINE_NAME}.yaml"
kfp.compiler.Compiler().compile(
    pipeline_func=my_pipeline,
    package_path=pipeline_filename)

with open(os.environ['KF_PIPELINES_SA_TOKEN_PATH'], "r") as f:
    TOKEN = f.read()

# Submit the pipeline to the KFP cluster
client = kfp.Client(host=KFP_HOST, # Use the configured KFP host
                    existing_token=TOKEN)  


client.create_run_from_pipeline_func(
    my_pipeline,
    enable_caching=False,
    arguments={
        'model_name': MODEL_NAME,
        'namespace': NAMESPACE,
        'model_uri': MODEL_URI
        
    })

#upload to Kubeflow 
# client.upload_pipeline(pipeline_package_path=pipeline_filename,
#                        pipeline_name="mlops",
#                        namespace = "kubeflow")

RunPipelineResult(run_id=a6b49222-05dd-427c-9826-1d42118355f2)

# Using Kserve component

## Manifest in var

In [ ]:
import kfp
from kfp import components
from kfp.dsl import component, pipeline, Input, Output, Dataset, Model, Metrics, ClassificationMetrics, OutputPath
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get GitHub username and token from environment variables
GITHUB_USERNAME = os.getenv("GITHUB_USERNAME")
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")

# Define configuration variables
REPO_URL = "https://github.com/danilonicioka/mlops-workflow.git"
CLONED_DIR = "mlops-workflow"
BRANCH_NAME = "tests"
PIPELINE_ID = "my-pipeline-id"
PIPELINE_NAME = "mlops"
KFP_HOST = "http://ml-pipeline.kubeflow.svc.cluster.local:8888"  # KFP host URL

# Define DVC remote configuration variables
REMOTE_NAME = "minio_remote"
REMOTE_URL = "s3://dvc-data"
MINIO_URL = "http://minio-svc.minio:9000"
ACCESS_KEY = os.getenv("ACCESS_KEY")
SECRET_KEY = os.getenv("SECRET_KEY")
DVC_FILE_DIR = 'data/external'
DVC_FILE_NAME = 'dataset.csv'

# Model serve config vars
MODEL_NAME = "youtubegoes5g"
NAMESPACE = "kubeflow-user-example-com"
SVC_ACCOUNT = "pipeline-runner"
MODEL_URI = "pvc://model-store-claim"

def create_serving_task(
    model_name, 
    namespace,
    model_uri
):
    api_version = "serving.kserve.io/v1beta1"
    serving_component_url = 'https://raw.githubusercontent.com/kubeflow/pipelines/master/components/kserve/component.yaml'

    inference_service = '''
apiVersion: {}
kind: "InferenceService"
metadata:
  name: {}
  namespace: {}
spec:
  predictor:
    model:
      modelFormat:
        name: pytorch
      storageUri: {}
'''.format(api_version, model_name, namespace, model_uri)

    serving_launcher_op = components.load_component_from_url(serving_component_url)
    serving_launcher_op(action="apply", inferenceservice_yaml=inference_service)
    
@pipeline
def my_pipeline(
    model_name: str,
    namespace: str,
    model_uri: str
):
    create_serving_task(model_name, namespace, model_uri)
    
# Compile the pipeline
pipeline_filename = f"{PIPELINE_NAME}.yaml"
kfp.compiler.Compiler().compile(
    pipeline_func=my_pipeline,
    package_path=pipeline_filename)

with open(os.environ['KF_PIPELINES_SA_TOKEN_PATH'], "r") as f:
    TOKEN = f.read()

# Submit the pipeline to the KFP cluster
client = kfp.Client(host=KFP_HOST, # Use the configured KFP host
                    existing_token=TOKEN)  

client.create_run_from_pipeline_func(
    my_pipeline,
    enable_caching=False,
    arguments={
        'model_name': MODEL_NAME,
        'namespace': NAMESPACE,
        'model_uri': MODEL_URI
    })

#upload to Kubeflow 
# client.upload_pipeline(pipeline_package_path=pipeline_filename,
#                        pipeline_name="mlops",
#                        namespace = "kubeflow")

## Setting vars

In [20]:
kserve_op = components.load_component_from_url(
    "https://raw.githubusercontent.com/kubeflow/pipelines/master/components/kserve/component.yaml"
)


@pipeline(name="KServe pipeline", description="A pipeline for KServe.")
def kservePipeline(
    action: str,
    model_name: str,
    model_uri: str,
    namespace: str,
    framework: str,
):
    kserve = kserve_op(
        action=action,
        model_name=model_name,
        model_uri=model_uri,
        namespace=namespace,
        framework=framework,
    )


# Compile the pipeline
pipeline_filename = f"{PIPELINE_NAME}.yaml"
kfp.compiler.Compiler().compile(
    pipeline_func=my_pipeline,
    package_path=pipeline_filename)

with open(os.environ['KF_PIPELINES_SA_TOKEN_PATH'], "r") as f:
    TOKEN = f.read()

# Submit the pipeline to the KFP cluster
client = kfp.Client(host=KFP_HOST, # Use the configured KFP host
                    existing_token=TOKEN)  

client.create_run_from_pipeline_func(
    kservePipeline,
    enable_caching=False,
    arguments={
        'action': "apply",
        'model_name': "youtubegoes5g",
        'model_uri': "pvc://model-store-claim",
        'namespace': "kubeflow-user-example-com",
        'framework': "pytorch"
    })

RunPipelineResult(run_id=1cf4b585-e78c-4a53-b28b-334b7579aff1)